In [1]:

from transformers import AutoTokenizer, AutoModelForCausalLM

import math
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3' 
# RUN ON CPU
# and do a time hack

/data2/jdownes/envs/summarization/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from huggingface_hub import login
login(token="<Enter your token here>")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/jdownes/.cache/huggingface/token
Login successful


In [3]:
i = open("test.txt", "r")
content = i.read()

i.close()

In [9]:

def _summarize(text,max_tokens, model, tokenizer):
    B_INST, E_INST = "[INST] ", " [/INST]"
    prompt = f"{B_INST}Write a concise summary of the following text:\n\n[TEXT_START]\n\n{text}\n\n[TEXT_END]\n\n{E_INST}"
   
    inputs = tokenizer( prompt,return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=max_tokens, use_cache=True, do_sample=True,temperature=0.2, top_p=0.95)

    prompt_length = inputs['input_ids'].shape[1]
    
    summary = tokenizer.decode(outputs[0][prompt_length:-1])
    
    return summary



    

In [10]:
# chunk tokens into list of texts
def chunk(tokens, max_token_length,tokenizer):
    token_length = len(tokens)
    k = math.ceil(token_length /max_token_length)
    chunk_sizes = [token_length // k + (1 if x < token_length % k else 0)  for x in range (k)]
    #print(token_length, k, chunk_sizes)
    last = 1
    texts =[]
    for l in chunk_sizes:
        sub_sequence_ids = tokens[last: last+l]
        last +=l
        texts.append(tokenizer.decode(sub_sequence_ids))
    return texts


In [11]:
def summarize(text, cache_dir = None):
    max_token_length = 1000

    tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", model_max_length = max_token_length, cache_dir= cache_dir)
    model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", pad_token_id = tokenizer.eos_token_id,cache_dir= cache_dir, device_map="auto")
    model.half()
    
    tokens = tokenizer.encode( text)#,return_tensors="pt")
    summary = ""
    complete_runthroughs = 0
    while len(tokens) > max_token_length:
        texts = chunk(tokens, max_token_length, tokenizer)
        
        summaries = []
        for text in texts:
            sub_summary = _summarize(text, max_token_length, model, tokenizer)
            summaries.append(sub_summary)
        summary = " ".join(summaries)
            
        tokens = tokenizer.encode(summary)
        complete_runthroughs+=1
        print(f"run through entire text doc {complete_runthroughs} times")
    return _summarize(summary, model, tokenizer)

In [12]:
result = summarize(content, cache_dir = "/data2/models/")

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.95s/it]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Token indices sequence length is longer than the specified maximum sequence length for this model (19626 > 1000). Running this sequence through the model will result in indexing errors
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/data2/jdownes/envs/summarization/lib/python3.8/site-packages/transformers/generation/utils.py:1510: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make s

run through entire text doc 1 times


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


run through entire text doc 2 times


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


run through entire text doc 3 times


In [13]:
print(f"summary for test doc that is {content.count(' ')} words, Approx {round(content.count(' ') / 500)} pages")
print(f"reduced to approximately {result.count(' ')} words")
print(result)
with open("sample_out.txt","w") as f:
    f.write(result)




summary for test doc that is 10261 words, Approx 21 pages
reduced to approximately 536 words
The "war on terror" is a global military campaign initiated by the United States following the September 11 attacks, targeting militant Islamist movements and other groups. The conflict has caused significant displacement and death and has been criticized for its morality, tactics, and impact on civil liberties and human rights. The term "war on terror" persists in U.S. politics, despite efforts by the Obama administration to replace it with "Overseas Contingency Operation." The text provides an overview of several military operations conducted by the United States and its allies against terrorist groups in different regions of the world, including Operation Enduring Freedom in Afghanistan, Pacific Eagle Philippines, and Operation Enduring Freedom Trans Sahara. The text also mentions other military operations, such as the Battle of Ras Kamboni in Somalia and the use of drones by the CIA in Paki